In [1]:
import matplotlib.pyplot as plt
import numpy as np

from alns import ALNS, State

In [2]:
%matplotlib inline

In [3]:
SEED = 5432

# The cutting-stock problem

The [cutting-stock problem (CSP)](https://en.wikipedia.org/wiki/Cutting_stock_problem) is well-known problem in operations research. Many different formulations exist, e.g. for the one-, two-, and even three-dimensional case. Here we solve an instance of the one-dimensional problem, obtained from the data [here](http://www.math.tu-dresden.de/~capad/cpd-ti.html#1D). It is known that the optimal solution for this problem achieves a wastage of only 74.

In [4]:
OPTIMAL = 74

In [5]:
with open('640.csp') as file:
    data = file.readlines()

NUM_BEAMS = int(data[0])
BEAM_LENGTH = int(data[1])

# Beams to be cut from the available beams
BEAMS = [int(length)
         for datum in data[2:]
         for length, amount in [datum.strip().split()]
         for _ in range(int(amount))]

print("Total number of beams available:", NUM_BEAMS)
print("Each available beam is of length:", BEAM_LENGTH)

print("Number of beams to be cut (orders):", len(BEAMS))

Total number of beams available: 165
Each available beam is of length: 1000
Number of beams to be cut (orders): 180


## Operators and solution state

To use the ALNS meta-heuristic, we need to have destroy and repair operators that work on a proposed solution, and a way to describe such a solution in the first place. The ALNS package exposes the class ``State`` to describe a solution, with an ``objective()`` member that computes this solution's objective value. Using it, we may compute a simple initial solution, and then apply the ALNS algorithm.

## Solution state

In [6]:
class CspState(State):
    """
    Solution state for the CSP problem. It has one member, assignments, which
    is a list of length NUM_BEAMS. Each entry is another list, containing the
    ordered beams cut from this beam. Each such sublist must sum to less than
    or equal to BEAM_LENGTH.
    """
    
    def __init__(self, assignments):
        self.assignments = assignments
    
    def copy(self):
        """
        Helper method to ensure each solution state is immutable.
        """
        return CspState(self.assignments.copy())

    def objective(self):
        """
        Computes the objective values, as the 'wasted' parts of each used 
        beam.
        """
        return sum(BEAM_LENGTH - sum(assignment) 
                   for assignment in self.assignments
                   if len(assignment) != 0)
    
    def plot(self):
        """
        Helper method to plot a solution.
        """
        lengths = [sum(assignment)
                   for assignment in self.assignments]
        
        fig, ax = plt.subplots(figsize=(12, 6))
      
        ax.barh(np.arange(len(self.assignments)), 
                lengths, 
                height=1)

        ax.set_xlim(right=BEAM_LENGTH)
        ax.set_yticks(np.arange(len(self.assignments), step=10))

        ax.margins(x=0, y=0)

        ax.set_xlabel('Usage')
        ax.set_ylabel('Beam (#)')

        plt.draw_if_interactive()

# Destroy operators

# Repair operators

# Initial solution